In [47]:
import pandas as pd
import numpy as np
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
load_dotenv()
import warnings
warnings.filterwarnings("ignore")
 
api_key = os.getenv('OPENAI_API_KEY')
model = AzureChatOpenAI(temperature=0, deployment_name="chat")
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [48]:
document = PyPDFLoader("../Resources/GCP.pdf")
data = document.load()
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        length_function = len,
        chunk_overlap = 250
)

documents = text_splitter.split_documents(data)

In [67]:
# Importación del pickle
data_embeddings = pd.read_pickle("../Resources/data_embeddings.pickle")
data_embeddings = pd.DataFrame(data_embeddings)

In [45]:
# Primera función
def question_user(q:str, embeddings=embedding_function):
  data_question = pd.DataFrame()
  emb = []
  q_list = []
  emb.append(embeddings.embed_query(q))
  q_list.append(q)
  data_question["pregunta"] = q_list
  data_question["embedding_pregunta"] = emb

  return data_question

def distance_vector(x, y):
    return np.dot(np.array(x), np.array(y))



# tercera función
def filtered_docs(data:object, p:str, param:float):
    data_sorted = data.sort_values(by = "distancia",ascending=False)
    data_sorted = data_sorted[data_sorted["distancia"] >= param]
    content = data_sorted["metadata"].tolist()
    return content, p

# Cuarta función
def pages_number(pags:list):
    list_pages = []
    for pag in range(len(pags)):
        list_pages.append(pags[pag]["page"])
    list_pages = list(set(list_pages))
    return list_pages

# Quinta función
def documents_prompt(documents:list, pages:list):
    docs = []
    for doc in range(len(documents)):
        val_aux = documents[doc].metadata["page"]
        if val_aux in pages:
            docs.append(documents[doc])
        else:
            continue
    return docs




In [68]:
# Example
question = "y para cloud storage?"
data_question = question_user(question)
data_embeddings["distancia"] = data_embeddings["embeddings"].apply(lambda x: distance_vector(x,data_question.iloc[0,1]))


In [69]:
data_embeddings = data_embeddings.sort_values(by="distancia", ascending=False)
data_embeddings.head(2)

,page_content,metadata,embeddings,distancia
227,Storage 31\nFundamental characteristics of an ...,"{'source': '../Resources/GCP.pdf', 'page': 40}","[-0.0017024035332724452, 0.01007207203656435, ...",0.649550
230,You can use cloud storage to store and serve s...,"{'source': '../Resources/GCP.pdf', 'page': 40}","[-0.05700239539146423, -0.009095244109630585, ...",0.634676


In [70]:
pages, q = filtered_docs(data_embeddings, question, 0.60)

In [71]:
pages = pages_number(pages)
pages

[39, 40, 104, 44, 52]

In [74]:
docs_ = documents_prompt(documents, pages)
docs_[:10]

[Document(page_content='CHAPTER TWO\nC loud storage is a simple, reliable, and scalable method for easily storing, retrieving, \nand sharing data in the cloud. Cloud providers offer managed data storage services \nthat eliminate the need for buying and managing your own storage infrastructure. Cloud \nstorage gives you the flexibility to scale your storage infrastructure as needed, globally \nand durably.Storage\n30', metadata={'source': '../Resources/GCP.pdf', 'page': 39}),
 Document(page_content='Storage\u200331\nFundamental characteristics of an effective cloud storage ser -\nvice include:\n• Security\xa0—  Data is stored securely (encrypted at rest and in transit).\n• Durability\xa0—  Data is stored redundantly, so it is not lost in the event of a  \ndisruption.\n• Availability\xa0—  Data is available at all times, whenever it’s needed.\nCommon cloud storage use cases include:\n• Compliance and business continuity\xa0 —  You can use cloud storage', metadata={'source': '../Resources

In [65]:
from langchain import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory


template = """
Historial de la conversación: {chat_history}
Tienes la siguiente información {context}
y debes responder la siguiente pregunta {human_input}.
Responde en español
"""

prompt = PromptTemplate(
        input_variables = ["context","question"],
        template=template
)
from langchain.chains.question_answering import load_qa_chain
llm_qa_documents = load_qa_chain(
    model,
    chain_type="stuff",
    prompt=prompt,
    verbose=True,
    memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history", input_key="human_input")
)


In [73]:
llm_qa_documents.run(input_documents=docs_[:10], human_input=question)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Historial de la conversación: Human: Que es BigQuery y cuales son sus casos de uso?
AI: BigQuery es un servicio de análisis de datos en la nube proporcionado por Google Cloud. Es una base de datos escalable y completamente administrada que permite realizar consultas rápidas y complejas sobre grandes conjuntos de datos. 

Algunos casos de uso de BigQuery incluyen:

1. Análisis de datos en tiempo real: BigQuery permite realizar consultas en tiempo real sobre datos en constante actualización, lo que lo hace ideal para aplicaciones que requieren análisis en tiempo real, como paneles de control y monitoreo de eventos.

2. Análisis de datos masivos: BigQuery está diseñado para manejar grandes volúmenes de datos, lo que lo convierte en una opción adecuada para empresas que necesitan analizar grandes conjuntos de datos históricos o en tiempo real.

3. Ciencia de datos y aprendizaje automá

'Cloud Storage es un servicio de almacenamiento en la nube que proporciona una forma simple, confiable y escalable de almacenar, recuperar y compartir datos en la nube. Algunos casos de uso comunes de Cloud Storage incluyen:\n\n1. Cumplimiento y continuidad del negocio: puedes utilizar el almacenamiento en la nube para realizar copias de seguridad y recuperación de datos, lo que te ayuda a cumplir con los requisitos de cumplimiento y garantizar la continuidad del negocio.\n\n2. Data lakes: puedes utilizar data lakes basados en almacenamiento en la nube para almacenar información en su forma cruda/nativa con metadatos para un acceso selectivo y análisis de datos.\n\n3. Desarrollo de aplicaciones: el almacenamiento es fundamental para el desarrollo y prueba de aplicaciones. Puedes utilizar el almacenamiento en la nube para almacenar y servir archivos estáticos (para sitios web) de forma global, crear aplicaciones rápidas e implementar sistemas de archivos compartidos entre equipos.\n\nCl